<a href="https://colab.research.google.com/github/shankarpm/GenAI/blob/main/AI_video_Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests torch torchvision transformers diffusers moviepy


In [ ]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!pip install diffusers transformers accelerate moviepy torchaudio audiocraft

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import os
from moviepy.editor import ImageSequenceClip, AudioFileClip
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from moviepy.editor import ImageSequenceClip, AudioFileClip
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from moviepy.editor import ImageSequenceClip, AudioFileClip, VideoFileClip # Import VideoFileClip

# Load the Stable Diffusion model (text-to-image)
def load_image_model():
    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
    pipe = pipe.to("cuda")  # If you have GPU support, use CUDA
    return pipe

# Generate frames using text prompt
def generate_frames(pipe, prompt, num_frames=10):
    frames = []
    for i in range(num_frames):
        image = pipe(prompt).images[0]
        frames.append(image)
    return frames

# Load MusicGen model (text-to-music)
def load_music_model():
    model = MusicGen.get_pretrained('melody')
    model.set_generation_params(duration=10)  # 10 seconds of music
    return model

# Generate music from a text prompt
def generate_music(model, prompt):
    print(f"Generating music for prompt: {prompt}")
    wav = model.generate([prompt])
    output_file = "output_music.wav"
    audio_write("output_music", wav[0], model.sample_rate, format="wav")
    return output_file

# Create a video from frames
def create_video_from_frames(frames, output_file="output_video.mp4", fps=2):
    frame_paths = [f'frame_{i}.png' for i, frame in enumerate(frames)]

    # Save frames
    for i, frame in enumerate(frames):
        frame.save(frame_paths[i])

    # Create video from image sequence
    clip = ImageSequenceClip(frame_paths, fps=fps)
    clip.write_videofile(output_file, fps=fps)

    return output_file

# Add music to video
# def add_music_to_video(video_path, music_path, output_with_audio="final_output_with_music.mp4"):
#     video_clip = ImageSequenceClip([video_path], fps=24)
#     audio_clip = AudioFileClip(music_path)

#     # Set the audio to the video
#     video_with_audio = video_clip.set_audio(audio_clip)

#     # Write the final video file with music
#     video_with_audio.write_videofile(output_with_audio, codec="libx264", audio_codec="aac")
#     return output_with_audio

# Add music to video
def add_music_to_video(video_path, music_path, output_with_audio="final_output_with_music.mp4"):
    # Get the frames from the video path (instead of creating new ImageSequenceClip)
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(music_path)

    # Set the audio to the video
    video_with_audio = video_clip.set_audio(audio_clip)

    # Write the final video file with music
    video_with_audio.write_videofile(output_with_audio, codec="libx264", audio_codec="aac")
    return output_with_audio

# Main function to generate video with music
def generate_video_with_music(image_prompt, music_prompt):
    # Generate images (frames)
    print("Loading image generation model...")
    image_pipe = load_image_model()
    print("Generating frames...")
    frames = generate_frames(image_pipe, image_prompt, num_frames=10)

    # Generate music
    print("Loading music generation model...")
    music_model = load_music_model()
    print("Generating music...")
    music_file = generate_music(music_model, music_prompt)

    # Create video
    print("Creating video from frames...")
    video_file = create_video_from_frames(frames)

    # Add music to video
    print("Adding music to video...")
    final_video = add_music_to_video(video_file, music_file)

    print(f"Final video with music created: {final_video}")

# Example usage
if __name__ == "__main__":
    image_prompt = "donald trump dancing in beach for espresso song"
    music_prompt = "espresso song by sabrina carpenter"
    generate_video_with_music(image_prompt, music_prompt)


Loading image generation model...


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  warnings.warn(



Generating frames...


...

  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")



Generating music...
Generating music for prompt: espresso song by sabrina carpenter
Creating video from frames...
Moviepy - Building video output_video.mp4.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
Adding music to video...
Moviepy - Building video final_output_with_music.mp4.
MoviePy - Writing audio in final_output_with_musicTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_output_with_music.mp4



Moviepy - Done !
Moviepy - video ready final_output_with_music.mp4
Final video with music created: final_output_with_music.mp4
